In [1]:
!pip install ptflops

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, confusion_matrix, accuracy_score)
from ptflops import get_model_complexity_info

# ROBUST PATH DISCOVERY

In [3]:
def find_paths():
    input_root = '/kaggle/input/'
    csv_path, img_dir = None, None
    
    # Walk through all directories in /kaggle/input
    for root, dirs, files in os.walk(input_root):
        if 'metadata.csv' in files:
            csv_path = os.path.join(root, 'metadata.csv')
        if 'images' in dirs:
            test_img_path = os.path.join(root, 'images')
            # Check if there are jpgs inside to confirm it's the right images folder
            if any(fname.endswith('.jpg') for fname in os.listdir(test_img_path)):
                img_dir = test_img_path
                
    if not csv_path or not img_dir:
        raise FileNotFoundError(f"Could not find metadata.csv or images folder in {input_root}")
    return csv_path, img_dir

CSV_FILE, IMG_DIR = find_paths()
print(f"Found Metadata: {CSV_FILE}")
print(f"Found Images: {IMG_DIR}")

Found Metadata: /kaggle/input/datasets/deadcardassian/pm25vision/train/metadata.csv
Found Images: /kaggle/input/datasets/deadcardassian/pm25vision/train/images


# SETTINGS & PATHS

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CSV_FILE = "/kaggle/input/datasets/deadcardassian/pm25vision/train/metadata.csv"
IMG_DIR = "/kaggle/input/datasets/deadcardassian/pm25vision/train/images"
INPUT_SIZE = (224, 224)
EPOCHS = 50 
BACKBONES = ["mobilenet_v2", "resnet50", "inception_v3", "efficientnet_b0", "resnet101"]
SPLITS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# DATA CLEANING & SAFETY

In [5]:
df_master = pd.read_csv(CSV_FILE)


min_label = df_master.iloc[:, 1].min()
if min_label > 0:
    df_master.iloc[:, 1] = df_master.iloc[:, 1] - min_label


df_master.iloc[:, 1] = df_master.iloc[:, 1].clip(0, 5)

# Filter for stratification safety
class_counts = df_master.iloc[:, 1].value_counts()
to_keep = class_counts[class_counts >= 10].index
df_master = df_master[df_master.iloc[:, 1].isin(to_keep)].reset_index(drop=True)

# 2. DATASET & STRATEGY

In [6]:
class PM25Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        # Filename handling
        raw_name = str(self.df.iloc[idx, 0])
        if not raw_name.lower().endswith('.jpg'): raw_name += '.jpg'
        img_name = os.path.join(IMG_DIR, os.path.basename(raw_name))
        
        image = Image.open(img_name).convert('RGB')
        
        # Label handling: Ensure it is a long integer for the loss function
        label = int(self.df.iloc[idx, 1])
        
        if self.transform: image = self.transform(image)
        return image, label

# 3. MODEL INITIALIZATION

In [7]:
def get_model(name):
    # Output is set to 6 classes (0, 1, 2, 3, 4, 5)
    if name == "mobilenet_v2":
        m = models.mobilenet_v2(weights='IMAGENET1K_V1')
        m.classifier[1] = nn.Linear(m.last_channel, 6)
    elif name == "resnet50":
        m = models.resnet50(weights='IMAGENET1K_V1')
        m.fc = nn.Linear(m.fc.in_features, 6)
    elif name == "efficientnet_b0":
        m = models.efficientnet_b0(weights='IMAGENET1K_V1')
        m.classifier[1] = nn.Linear(m.classifier[1].in_features, 6)
    elif name == "inception_v3":
        m = models.inception_v3(weights='IMAGENET1K_V1', aux_logits=True)
        m.fc = nn.Linear(m.fc.in_features, 6)
    elif name == "resnet101":
        m = models.resnet101(weights='IMAGENET1K_V1')
        m.fc = nn.Linear(m.fc.in_features, 6)
    return m.to(device)

# 4. MAIN EXECUTION LOOP

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("✅ Transform defined. You can now run the MobileNetV2 cell.")

✅ Transform defined. You can now run the MobileNetV2 cell.


# Optimized MobileNetV2

In [9]:
from torch.cuda.amp import GradScaler, autocast


BATCH_SIZE = 64 

# 2. Setup Mixed Precision Scaler
scaler = GradScaler()

mobilenet_results = []

for test_sz in SPLITS:
    split_label = f"{int((1-test_sz)*100)}:{int(test_sz*100)}"
    print(f"🚀 Training: MobileNetV2 | Split: {split_label} | Epochs: {EPOCHS}")
    
    
    model = models.mobilenet_v2(weights='IMAGENET1K_V1')
    model.classifier[1] = nn.Linear(model.last_channel, 6)
    model = model.to(device)
    
    train_val, test_df = train_test_split(df_master, test_size=test_sz, stratify=df_master.iloc[:,1], random_state=42)
    train_df, val_df = train_test_split(train_val, test_size=0.10, stratify=train_val.iloc[:,1], random_state=42)
    
    # Speed Optimization: Added num_workers and pin_memory
    train_ldr = DataLoader(PM25Dataset(train_df, transform), batch_size=BATCH_SIZE, 
                           shuffle=True, num_workers=2, pin_memory=True)
    test_ldr = DataLoader(PM25Dataset(test_df, transform), batch_size=BATCH_SIZE, 
                          num_workers=2, pin_memory=True)
    
    opt = optim.Adam(model.parameters(), lr=1e-4)
    crit = nn.CrossEntropyLoss()
    
    start_t = time.time()
    for epoch in range(50): # Kept at 50 per instructions
        model.train()
        for imgs, lbls in train_ldr:
            imgs, lbls = imgs.to(device), lbls.to(device)
            opt.zero_grad()
            
            # Use autocast for faster math
            with autocast():
                out = model(imgs)
                if hasattr(out, 'logits'): out = out.logits # Safety for Inception
                loss = crit(out, lbls)
            
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
    
    
    model.eval(); y_t, y_p = [], []
    with torch.no_grad():
        for imgs, lbls in test_ldr:
            imgs, lbls = imgs.to(device), lbls.to(device)
            with autocast():
                preds = model(imgs).argmax(1)
            y_t.extend(lbls.cpu().numpy())
            y_p.extend(preds.cpu().numpy())
    
    train_time = time.time() - start_t
    mobilenet_results.append({
        "Model": "MobileNetV2", "Split": split_label, 
        "F1": f1_score(y_t, y_p, average='macro'), "Time": train_time
    })
    
    # Save after each split 
    pd.DataFrame(mobilenet_results).to_csv("mobilenet_results.csv", index=False)
    
   
    del model, opt, train_ldr, test_ldr
    torch.cuda.empty_cache()

print("Finished MobileNetV2 successfully.")

🚀 Training: MobileNetV2 | Split: 90:10 | Epochs: 50
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


/tmp/ipykernel_24/2493415534.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
100%|██████████| 13.6M/13.6M [00:00<00:00, 225MB/s]
/tmp/ipykernel_24/2493415534.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/2493415534.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: MobileNetV2 | Split: 80:20 | Epochs: 50


/tmp/ipykernel_24/2493415534.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/2493415534.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: MobileNetV2 | Split: 70:30 | Epochs: 50


...

🚀 Training: MobileNetV2 | Split: 19:80 | Epochs: 50


/tmp/ipykernel_24/2493415534.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/2493415534.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: MobileNetV2 | Split: 9:90 | Epochs: 50


/tmp/ipykernel_24/2493415534.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/2493415534.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Finished MobileNetV2 successfully.


# Optimized ResNet50

In [10]:
# 1. Initialize ResNet50
resnet50_results = []

for test_sz in SPLITS:
    split_label = f"{int((1-test_sz)*100)}:{int(test_sz*100)}"
    print(f"🚀 Training: ResNet50 | Split: {split_label} | Epochs: {EPOCHS}")
    
    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, 6)
    model = model.to(device)
    
    train_val, test_df = train_test_split(df_master, test_size=test_sz, stratify=df_master.iloc[:,1], random_state=42)
    train_df, val_df = train_test_split(train_val, test_size=0.10, stratify=train_val.iloc[:,1], random_state=42)
    
    train_ldr = DataLoader(PM25Dataset(train_df, transform), batch_size=BATCH_SIZE, 
                           shuffle=True, num_workers=2, pin_memory=True)
    test_ldr = DataLoader(PM25Dataset(test_df, transform), batch_size=BATCH_SIZE, 
                          num_workers=2, pin_memory=True)
    
    opt = optim.Adam(model.parameters(), lr=1e-4)
    crit = nn.CrossEntropyLoss()
    
    start_t = time.time()
    for epoch in range(50):
        model.train()
        for imgs, lbls in train_ldr:
            imgs, lbls = imgs.to(device), lbls.to(device)
            opt.zero_grad()
            with autocast():
                out = model(imgs)
                loss = crit(out, lbls)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
    
    model.eval(); y_t, y_p = [], []
    with torch.no_grad():
        for imgs, lbls in test_ldr:
            imgs, lbls = imgs.to(device), lbls.to(device)
            with autocast():
                preds = model(imgs).argmax(1)
            y_t.extend(lbls.cpu().numpy()); y_p.extend(preds.cpu().numpy())
    
    resnet50_results.append({
        "Model": "ResNet50", "Split": split_label, 
        "F1": f1_score(y_t, y_p, average='macro'), "Time": time.time() - start_t
    })
    
    pd.DataFrame(resnet50_results).to_csv("resnet50_results.csv", index=False)
    del model, opt, train_ldr, test_ldr
    torch.cuda.empty_cache()

print("Finished ResNet50 successfully.")

🚀 Training: ResNet50 | Split: 90:10 | Epochs: 50
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 237MB/s]
/tmp/ipykernel_24/3477948454.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/3477948454.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: ResNet50 | Split: 80:20 | Epochs: 50


/tmp/ipykernel_24/3477948454.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/3477948454.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: ResNet50 | Split: 70:30 | Epochs: 50


...

🚀 Training: ResNet50 | Split: 19:80 | Epochs: 50


/tmp/ipykernel_24/3477948454.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/3477948454.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🚀 Training: ResNet50 | Split: 9:90 | Epochs: 50


/tmp/ipykernel_24/3477948454.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_24/3477948454.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Finished ResNet50 successfully.
